In [ ]:
import http.client
import json
import csv
import time
import os
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
API_KEY = os.getenv("SCRAPTIK_API_KEY")
USER_ID = os.getenv("TIKTOK_USER_ID")

def log_error(message):
    with open("task_error_log.txt", "a", encoding="utf-8") as log_file:
        log_file.write(f"[{datetime.now()}] {message}\n")

def fetch_user_posts(api_key, user_id):
    conn = http.client.HTTPSConnection("scraptik.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': api_key,
        'x-rapidapi-host': "scraptik.p.rapidapi.com"
    }

    max_cursor = 0
    has_more = True
    existing_posts = load_existing_posts()

    while has_more:
        try:
            conn.request("GET", f"/user-posts?user_id={user_id}&count=50&max_cursor={max_cursor}", headers=headers)
            res = conn.getresponse()
            status = res.status
            data = res.read()

            if status == 429:
                print("🚫 Rate limit exceeded!")
                log_error("Rate limit exceeded")
                break
            elif status != 200:
                print(f"⚠️ API error: {status}")
                log_error(f"API error: {status}")
                break

            json_data = json.loads(data.decode("utf-8"))

            if 'aweme_list' in json_data:
                new_posts = json_data['aweme_list']
                has_more = json_data.get('has_more', False)
                max_cursor = json_data.get('max_cursor', 0)

                for post in new_posts:
                    post_id = post.get('aweme_id', '')
                    if post_id:
                        post_data = [
                            post.get('aweme_id', ''),
                            post.get('desc', ''),
                            post['statistics'].get('digg_count', 0),
                            post['statistics'].get('comment_count', 0),
                            post['statistics'].get('share_count', 0),
                            post['statistics'].get('play_count', 0),
                            post['video']['play_addr']['url_list'][0] if post.get('video') else '',
                            datetime.utcfromtimestamp(post.get('create_time', 0)).strftime('%Y-%m-%d %H:%M:%S'),
                            post['video'].get('duration', 0) if post.get('video') else ''
                        ]
                        existing_posts[post_id] = post_data

                time.sleep(1)
            else:
                print("⚠️ Unexpected response format.")
                log_error("Unexpected response format")
                break

        except Exception as e:
            log_error(f"Error fetching data: {e}")
            break

    save_posts_to_csv(existing_posts)
    print(f"✅ {len(existing_posts)} posts written to Debbhie_tiktok.csv")

def load_existing_posts():
    posts = {}
    if os.path.exists('Debbhie_tiktok.csv'):
        with open('Debbhie_tiktok.csv', mode='r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            header = next(reader)
            for row in reader:
                post_id = row[0]
                posts[post_id] = row
    return posts

def save_posts_to_csv(posts):
    with open('Debbhie_tiktok.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Post ID', 'Description', 'Likes', 'Comments', 'Shares', 'Views', 'Video URL', 'Date', 'Duration'])
        for post in posts.values():
            writer.writerow(post)

if __name__ == "__main__":
    if API_KEY and USER_ID:
        fetch_user_posts(API_KEY, USER_ID)
    else:
        print("❌ Please set your SCRAPTIK_API_KEY and TIKTOK_USER_ID in the .env file.")
